In [6]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from ipywidgets import widgets
%matplotlib widget

def mask_overlay(image, mask, color=(0., 1., 0.), weight=.8):
    """
    Helper function to visualize mask on the top of the aneurysm
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = image * weight + mask * (1 - weight) # cv2.addWeighted(image, 1 - weight, mask, weight, 0.,  dtype=cv2.CV_32F)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]
    return img


class ImageSlicer(object):

    def __init__(self, ax, img, mask, start_index):
        self.ax = ax

        # convert to numpy array
        self.image_np = img
        self.mask_np = mask

        # get number of slices
        _, _, self.slices = self.image_np.shape
        self.ind = start_index if start_index is not None else self.slices // 2
        self.show_mask = True

        # plot image with mask overlay
        self.image_plt = self.ax.imshow(self.overlay)
        self._draw()

    @property
    def overlay(self):
        # get image and mask slice
        image = self.image_np[:, :, self.ind]
        image = image / np.max(image)
        image = np.dstack((image, image, image))
        mask = self.mask_np[:, :, self.ind]

        # create masked overlay
        if self.show_mask:
            return mask_overlay(image, mask)
        else: 
            return image

    def onscroll(self, event):

        # get new slice number
        self.ind = event['new']
        self.update()

    def on_show_mask(self, event):

        self.show_mask = event['new']
        with open("file.txt", "a") as f:
            f.write(f"{self.show_mask}\n")
        self.update()

    def update(self):

        # draw overlay
        self.image_plt.set_data(self.overlay)
        self._draw()

    def _draw(self):
        try:
            self.image_plt.axes.figure.canvas.draw()
        except Exception as e:
            with open("file.txt", "a") as f:
                f.write(f"Exception {e}\n")
            pass


def plot3d(img, mask, start_index = None):

    #
    figure, ax = plt.subplots(1, 1)
    tracker = ImageSlicer(ax, img, mask, start_index)

    #
    int_slider = widgets.IntSlider(
        value=tracker.ind,
        min=0,
        max=tracker.slices,
        step=1,
        description='Slice',
        continuous_update=True
    )
    int_slider.observe(tracker.onscroll, 'value')

    show_mask = widgets.Checkbox(
        value=True,
        description='Show Mask',
        disabled=False,
        indent=False
    )
    show_mask.observe(tracker.on_show_mask, 'value')

    return figure, int_slider, show_mask

def load_case(i):
    f = h5py.File(f"./data/val/{files[i]}", "r")
    return f["raw"][:], f["label"][:]

In [3]:
import yaml
import os
from pytorch3dunet.augment.transforms import Transformer

config_path = "./train-configs/aug_tests/id.yml"

config = yaml.safe_load(open(config_path, 'r'))

transform_config = config["loaders"]["train"]["transformer"]
datapath = config["loaders"]["train"]["file_paths"][0]

transformer = Transformer(transform_config, {})
raw_transform = transformer.raw_transform()
label_transform = transformer.label_transform()

files = os.listdir("./data/val")
print(files)

['A133.h5', 'A027.h5', 'A096_L.h5', 'A003.h5', 'A085.h5', 'A137.h5', 'A066.h5', 'A080.h5', 'A084.h5', 'A033.h5', 'A028.h5', 'A029.h5', 'A134.h5', 'A008.h5', 'A049.h5', 'A056.h5', 'A014.h5', 'A059_R.h5', 'A103.h5', 'A078_R.h5', 'A077.h5', 'A105_R.h5', 'A012.h5', 'A047.h5', 'A019.h5', 'A118.h5', 'A070.h5', 'A032.h5', 'A043.h5', 'A097.h5', 'A031.h5', 'A018.h5', 'A072.h5']


In [98]:
from scipy.ndimage import percentile_filter, binary_dilation

padding = 40
def pad_min_max(max, ival):
    return slice(
        np.clip(ival[0] - padding, a_min=0, a_max=max - 1),
        np.clip(ival[1] + padding, a_min=0, a_max=max - 1)
    )

def get_aneurysm_bounds(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = np.where(x_s)[0][[0, -1]]
    y = np.where(y_s)[0][[0, -1]]
    z = np.where(z_s)[0][[0, -1]]

    x = pad_min_max(mask.shape[0], x)
    y = pad_min_max(mask.shape[1], y)
    z = pad_min_max(mask.shape[2], z)

    return x, y, z

def check_percentile(low, high):


    
    return s / len(files)

start_low = 1
start_high = 90
r = 10

res = np.zeros((15, 15))
for file in files:
    print(file)
    with h5py.File(f"./data/train/{file}", "r") as f:
        gt = f["label"][:]
        raw = f["raw"][:]

        bounds = get_aneurysm_bounds(gt)

        gt_bounded = gt[bounds]
        raw_bounded = raw[bounds]

        ignore_mask = np.logical_not(binary_dilation(gt_bounded, iterations=5))
        gt_bounded[ignore_mask] = 0

        for low in range(start_low, start_low + r):
            for high in range(start_high, start_high + r):
                print(low, high)
                low_perc = np.percentile(raw, low)
                high_perc = np.percentile(raw, high)
                
                raw_copy = raw_bounded.copy()
                raw_copy[raw_copy < low_perc] = 0
                raw_copy[raw_copy > high_perc] = 1
                raw_copy[ignore_mask] = 0

                res[low - start_low, high - start_high] += np.mean(np.abs(gt_bounded - raw_copy))
    

        
res

A024.h5
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
3 90
3 91
3 92
3 93
3 94
3 95
3 96
3 97
3 98
3 99
4 90
4 91
4 92
4 93
4 94
4 95
4 96
4 97
4 98
4 99
5 90
5 91
5 92
5 93
5 94
5 95
5 96
5 97
5 98
5 99
6 90
6 91
6 92
6 93
6 94
6 95
6 96
6 97
6 98
6 99
7 90
7 91
7 92
7 93
7 94
7 95
7 96
7 97
7 98
7 99
8 90
8 91
8 92
8 93
8 94
8 95
8 96
8 97
8 98
8 99
9 90
9 91
9 92
9 93
9 94
9 95
9 96
9 97
9 98
9 99
10 90
10 91
10 92
10 93
10 94
10 95
10 96
10 97
10 98
10 99
A096_R.h5
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
3 90
3 91
3 92


KeyboardInterrupt: 

In [ ]:
res[:10, :10].argmin()
t = res[:10, :10]
m = np.unravel_index(t.argmin(), t.shape)
m[0] + start_low, m[1] + start_high

(10, 99)

In [4]:
padding = 50
def pad_min_max(max, ival):
    return slice(
        np.clip(ival[0] - padding, a_min=0, a_max=max - 1),
        np.clip(ival[1] + padding, a_min=0, a_max=max - 1)
    )

def get_aneurysm_bounds(mask):
    x_s = mask.sum(axis=(1, 2))
    y_s = mask.sum(axis=(0, 2))
    z_s = mask.sum(axis=(0, 1))

    x = np.where(x_s)[0][[0, -1]]
    y = np.where(y_s)[0][[0, -1]]
    z = np.where(z_s)[0][[0, -1]]

    x = pad_min_max(mask.shape[0], x)
    y = pad_min_max(mask.shape[1], y)
    z = pad_min_max(mask.shape[2], z)

    return x, y, z

In [20]:
import torch
f = h5py.File("./test/2800_0.h5", "r")
gt = f["label"][0, 0]
pred = f["pred"][0, 0]
raw = f["raw"][0, 0]


#pred = (torch.sigmoid(torch.from_numpy(pred)) > 0.5).long()


fig, slider = plot3d(raw, pred)
slider

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=64, description='Slice', max=128)

In [7]:
from scipy.ndimage import percentile_filter, binary_dilation, maximum_filter, minimum_filter, generic_filter
import pandas as pd
i = 0

img, mask = load_case(i)
img /= img.max()

# img_t = raw_transform(img)
# mask_t = label_transform(mask)

# img = img[:128, :128, 128]
# mask = mask[128, :128, 128]

# out_img = percentile_filter(img, size=3, percentile=50)

def transform(img, mask):
    high_perc = np.percentile(img, 99)
    img_b = img.copy()
    img_b[img_b >= high_perc] = 1
    img_b[img_b < high_perc] = 0
    img_b = minimum_filter(img_b, size=2)
    img_b = binary_dilation(img_b, iterations=2)

    # img[mask_e != 1] = 0

    return img_b, mask

def calc_overlap(img_b, mask):
    return np.mean(img_b[mask == 1])

ret = []
ret_2 = []

for i in range(len(files)):
    img, mask = load_case(i)
    img /= img.max()
    img_b, mask = transform(img, mask)
    ret.append(calc_overlap(img_b, mask))
    ret_2.append(img_b.sum() / (img_b.shape[0] * img_b.shape[1] * img_b.shape[2]))

s = pd.Series(ret)
s_2 = pd.Series(ret_2)
print(s.describe())
print(s_2.describe())


# bounds = get_aneurysm_bounds(mask)

# zero_mask = np.zeros(img_b.shape)
# print(calc_overlap(img_b, mask))

# fig, slider, show_mask = plot3d(img_b[bounds], mask[bounds])

# widgets.Box(
#     [slider, show_mask]
# )


count    33.000000
mean      0.964741
std       0.029528
min       0.866838
25%       0.949482
50%       0.970121
75%       0.986667
max       1.000000
dtype: float64
count    33.000000
mean      0.013147
std       0.002039
min       0.008905
25%       0.012248
50%       0.013932
75%       0.014500
max       0.016158
dtype: float64


In [ ]:
Coverage 1 weniger
count    75.000000
mean      0.850895
std       0.067300
min       0.658271
25%       0.818405
50%       0.867841
75%       0.898143
max       0.942187

Coverage
count    75.000000
mean      0.997276
std       0.008810
min       0.931575
25%       0.997698
50%       0.999767
75%       1.000000
max       1.000000

% of input
dtype: float64
count    75.000000
mean      0.020444
std       0.003271
min       0.009217
25%       0.018929
50%       0.020931
75%       0.022494
max       0.025825
dtype: float64